# LaB-GATr Basic Usage

Before using this notebook, install the correct dependencies and LaB-GATr as follows:

Optional new Anaconda environment
```
conda create --name lab-gatr python=3.10
conda activate lab-gatr
``` 
Next, install PyTorch and xFormers and other libraries
```
pip install torch==2.1.0 --index-url https://download.pytorch.org/whl/cu121
pip install xformers==0.0.22.post7 --index-url https://download.pytorch.org/whl/cu121
pip install torch_geometric==2.4.0
pip install torch_scatter torch_cluster -f https://data.pyg.org/whl/torch-2.1.0+cu121.html
```
Install LaB-GATr itself, which also install GATr
```
pip install .
```
Additionally, if you have made a new Anaconda environment, install Jupyter
```
pip install jupyter jupyterlab
```


In [1]:
import torch
from lab_gatr import PointCloudPoolingScales, LaBGATr
import torch_geometric as pyg
from gatr.interface import embed_oriented_plane, extract_translation

/home/ajay/anaconda3/envs/anatomygen_gatr/lib/python3.10/site-packages/gatr/primitives/bilinear.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sparse_basis = torch.loa

Let us first create a dummy mesh: n positions and orientations (e.g. surface normal) and an arbitrary scalar feature (e.g. geodesic distance).

In [2]:
n = 1000

pos, orientation = torch.rand((n, 3)), torch.rand((n, 3))
scalar_feature = torch.rand(n)

Next, a point cloud pooling transform for the tokenisation (patching). Also apply this as a transform to create the dummy data.

In [3]:
transform = PointCloudPoolingScales(rel_sampling_ratios=(0.2,), interp_simplex='triangle')
dummy_data = transform(pyg.data.Data(pos=pos, orientation=orientation, scalar_feature=scalar_feature))

A geometric algebra interface to embed your data in $\mathbf{G}(3, 0, 1)$.

In [4]:
class GeometricAlgebraInterface:
    num_input_channels = num_output_channels = 1
    num_input_scalars = num_output_scalars = 1

    @staticmethod
    @torch.no_grad()
    def embed(data):

        multivectors = embed_oriented_plane(normal=data.orientation, position=data.pos).view(-1, 1, 16)
        scalars = data.scalar_feature.view(-1, 1)

        return multivectors, scalars

    @staticmethod
    def dislodge(multivectors, scalars):
        return extract_translation(multivectors).squeeze()


Create a model instance with dimensionality 8, 10 GATr blocks, and 4 attention heads in each block. It uses the geometric algebra interface we defined above.

In [5]:
model = LaBGATr(GeometricAlgebraInterface, d_model=8, num_blocks=10, num_attn_heads=4, use_class_token=False)

LaB-GATr (261761 parameters)


Generate some output with the dummy data to verify that the model functions. Training or inference from here on is the same as any PyTorch model.

In [6]:
output = model(dummy_data)
print(output.shape)

torch.Size([1000, 3])


/home/ajay/anaconda3/envs/anatomygen_gatr/lib/python3.10/site-packages/gatr/primitives/bilinear.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sparse_basis = torch.loa